In [1]:
!pip install rank_bm25
!git clone https://github.com/cr-nlp/project1-2023.git

Cloning into 'project1-2023'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (8/8), 2.30 MiB | 2.90 MiB/s, done.


## Import Libraries

In [2]:
import urllib.request as re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Import des données

In [4]:
#Code prof Import des 3 fichiers

def loadNFCorpus():
	dir = "./project1-2023/"
	filename = dir +"dev.docs"

	dicDoc={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		#print(tabLine)
		key = tabLine[0]
		value = tabLine[1]
		#print(value)
		dicDoc[key] = value
	filename = dir + "dev.all.queries"
	dicReq={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		key = tabLine[0]
		value = tabLine[1]
		dicReq[key] = value
	filename = dir + "dev.2-1-0.qrel"
	dicReqDoc=defaultdict(dict)
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.strip().split('\t')
		req = tabLine[0]
		doc = tabLine[2]
		score = int(tabLine[3])
		dicReqDoc[req][doc]=score

	return dicDoc, dicReq, dicReqDoc

In [5]:
dicDoc, dicReq, dicReqDoc = loadNFCorpus()

## Mise en forme de la donnée, enlèvement des mots inutiles

In [6]:
# Mise en forme des article, enlèvement des mots inutiles
def text2TokenList(text):
	stopword = stopwords.words('english')
	#print("LEN DE STOPWORD=",len(stopword))
	word_tokens = word_tokenize(text.lower())
	word_tokens_without_stops = [word for word in word_tokens if word not in stopword and len(word)>2]

	return word_tokens_without_stops

In [7]:
docsToKeep=[]
reqsToKeep=[]
dicReqDocToKeep=defaultdict(dict)

#Construction du dictionnaire de Req à garder
i=0
for reqId in dicReqDoc:
    if i >= len(dicDoc) :  #nbDocsToKeep:
        break
    for docId in dicReqDoc[reqId]:
        dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
        docsToKeep.append(docId)
        i = i + 1
    reqsToKeep.append(reqId)
docsToKeep = list(set(docsToKeep))

## Construction des corpus des MED et PLAIN

In [8]:
allVocab ={}
#Construction corpus des MED
for k in docsToKeep: #Pour chaque MED
    docTokenList = text2TokenList(dicDoc[k]) #Transforme les texte de MED en liste
    #print(docTokenList)
    for word in docTokenList: #Pour chaque mot dans la liste
        if word not in allVocab: #S'il n'y est pas déjà
            allVocab[word] = word #l'ajoute dans le dictionnaire

allVocabListDoc = list(allVocab)
#print("doc vocab=",allVocabListDoc)


allVocab ={}

#Construction corpus des PLAIN
for k in reqsToKeep: #Pour chaque PLAIN
    docTokenList = text2TokenList(dicReq[k]) # Transforme les texte de PLAIN en liste
    #print(docTokenList)
    for word in docTokenList: # Pour chaque mot dans la liste
        if word not in allVocab: #S'il n'y est pas déjà
            allVocab[word] = word #l'ajoute dans le dictionnaire
allVocabListReq = list(allVocab)

In [9]:
corpusDocTokenList = {}
corpusReqTokenList = {}
corpusDocName=[]
corpusDicoDocName={}
i = 0

for k in docsToKeep: #Pour chaque MED
    docTokenList = text2TokenList(dicDoc[k]) #Transforme son corpus en liste
    corpusDocTokenList[k]=docTokenList #complete le dictionnaire des mots pour chaque MED
    corpusDocName.append(k) #Ajoute le nom du MED dasn sa propre liste
    corpusDicoDocName[k] = i # Attribue un numero a chaque MED
    i = i + 1

#print("reqs...")
corpusReqName=[]
corpusDicoReqName={}
i = 0
for k in reqsToKeep: # Pour tous les PLAIN
    reqTokenList = text2TokenList(dicReq[k]) #Transforme son corpus en liste
    corpusReqTokenList[k] = reqTokenList#complete le dictionnaire des mots pour chaque PLAIN
    corpusReqName.append(k) #Ajoute le nom du PLAIN dans sa propre liste
    corpusDicoReqName[k] = i# Attribue un numero a chaque PLAIN
    i = i + 1

## Recherche du score TF-IDF

In [10]:
%%time
TableTFIDF=pd.DataFrame(columns=['MED','PLAIN','score'])
c=0
CumulTFIDF=0
for plain in corpusReqTokenList: #Pour chaque mot dans le plain avec doublons
    j=0
    reqTokenList = corpusReqTokenList[plain]  #liste de mots de PLAIN n°req
    #print(req)
    #print(len(doc_scores)) #nombre de mot dans chaque PLAIN ?
    trueDocs = np.zeros(len(corpusDocTokenList)) #Ressort un array de la forme de corpusDocTokenList (1779,)
    for med in corpusDicoDocName: #pour chaque nom de MED
        if med in dicReqDocToKeep[plain]: # et si MED n°docId est dans les recommendation de PLAIN
            vectorizer=TfidfVectorizer() # Déclare un nouveau vecteur
            pos=corpusDicoDocName[med] # Recupère la position du MED
            textMED= ' '.join(corpusDocTokenList[med]) # Recrée un cropus str à partir de la liste de MED
            textPLAIN=' '.join(corpusReqTokenList[plain]) # Recrée un cropus str à partir de la liste de PLAIN
            score=vectorizer.fit_transform([textMED,textPLAIN]) # Cherche le score TF IDF des MED et PLAIN associés
            trueDocs[corpusDicoDocName[med]] = score.mean() # Somme des moyennes de TFIDF de MED pour PLAIN n°plain
    CumulTFIDF=CumulTFIDF + score.mean()
    c = c + 1
moytfidf=CumulTFIDF/c
print(moytfidf)



0.03777821460241738
CPU times: user 12 s, sys: 84.5 ms, total: 12.1 s
Wall time: 16.7 s
